In [3]:
id_w, eid2w, ew2id = 1, {}, {}
with open('train_en.vocab.4k', 'r', encoding='utf-8') as f:
    for w in f:
        w = w.strip()
        eid2w[id] = w
        ew2id[w] = id
        id +=1

ev = id

hello world2


In [ ]:
edata = []
with open('train.en', 'r', encoding='utf-8') as f:
    for sen in f:
        w1 = [ew2id['<s>']]
        for w in sen.strip().split():
            if w in ew2id:
                w1.append(ew2id[w])
            else:
                w1.append(ew2id['<unk>'])
        w1.append(ew2id['</s>'])
        edata.appned(w1)

In [ ]:
class MyNMT(nn.Module):
    def __init__(self, jv, k):
        super(MyNMT, self).__init__()
        self.jemb = nn.Embedding(jv, k)
        self.eemb = nn.Embedding(ev, k)
        self.lstm1 = nn.LSTM(k, k, num_layers=2)
        self.lstm2 = nn.LSTM(k, k, num_layers=2)
        self.W = nn.Linear(k, ev)
    def forward(self, jline, eline):
        x = self.jemb(jline)
        ox, (hnx, cnx) = self.lstm1(x)
        y = self.eemb(eline)
        oy, (hny, cny) = self.lstm2(y, (hnx, cnx))
        out = self.W(oy)
        return out
    

In [ ]:
net.train()
for epoch in range(20):
    loss1k = 0.0
    for i in range(len(jdata)):
        jinput = torch.LongTensor([jdata[i][1:]]).to(device)
        einput = torch.LongTensor([edata[i][:-1]]).to(device)
        out = net(jinput, einput)
        gans = torch.LongTensor([edata[i][1:]]).to(device)
        loss = criterion(out[0], gans[0])
        loss1k += loss.item()
        if (i%100 == 0):
            print(epoch, i, loss1k)
            loss1k = 0.0
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    outfile = "nmt-" + str(epoch) + ".model"
    torch.save(net.state_diact(), outfile)
    

In [ ]:
esid = ew2id['<s>']
eeid = ew2id['</s>']
net.eval()
with torch.no_grad():
    for i in range(len(jdata)):
        jinput = torch.LongTensor([jdata[i][1: ]]).to(device)
        x = net.jemb(jinput)
        ox, (hn, cn) = net.lstm1(x)
        wid = esim
        s1 = 0
        while True:
            wids = torch.LongTensor([[wid]]).to(device)
            y = net.eemb(wids)
            oy, (hn, cn) = net.lstm2(y, (hn, cn))
            oy1 = net.W(oy)
            wid = torch.argmax(F.softmax(ox1[0], dim=1)).item()
            if (wid==eeid):
                break
            print(eid2w[wid], " ", end='')
            s1 += 1
            if (s1==30):
                break
        

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
class MyAttNMT(nn.Module):
    def __init__(self, jv, ev, k):
        super(MyAttNMT, self).__init__()
        self.jemb = nn.Embedding(jv, k)
        self.eemb = nn.Embedding(ev, k)
        self.lstm1 = nn.LSTM(k, k, num_layers=2, batch_first=True)
        self.lstm2 = nn.LSTM(k, k, num_layers=2, batch_first=True)
        self.Wc = nn.Linear(2*k, k)
        self.W = nn.Linear(k, ev)
    def forward(self, jline, eline):
        x = self.jemb(jline)
        ox, (hnx, cnx) = self.lstm1(x)
        y = self.eemb(eline)
        oy, (hny, cny) = self.lstm2(y, (hnx, cnx))
        ox1 = ox.permute(0, 2, 1)
        sim = torch.bmm(oy, ox1)
        bs, yws, xws = sim.shape
        sim2 = sim.reshape(bs*yws, xws)
        alpha = F.softmax(sim2, dim=1).reshape(bs, yws, xws)
        ct = torch.bmm(alpha, ox)
        oy1 = torch.cat([ct, oy], dim=2)
        oy2 = self.Wc(oy1)
        return self.W(oy2)


In [ ]:
net.train()
for ep in range(20):
    i=0
    for xs, ys in dataloader:
        xs1, ys1, ys2 = [], [], []
        for k in range(len(xs)):
            tid = xs[k]
            xs1.append(torch.LongTensor(tid[1:]))
            tid = ys[k]
            ys1.append(torch.LongTensor(tid[:-1]))
            ys2.append(torch.LongTensor(tid[1:]))
        jinput = pad_sequence(xs1, batch_first=True).to(device)
        einput = pad_sequence(ys1, batch_first=True).to(device)
        gans = pad_sequence(ys2, batch_first=True, padding_value=-1.0).to(device)
        out = net(jinput, einput)
        loss = criterion(out[0], gans[0])
        for h in range(1, len(gans)):
            loss += criterion(out[h], gans[h])
        print(ep, i, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
    outfile = 'attnmt2-'+str(ep)+'.model'
    torch.save(net.state_dict(), outfile)